<a href="https://colab.research.google.com/github/Reptilefury/coursera-machine-learning/blob/main/Intro_for_experts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
#Load the data set 
mnist = tf.keras.datasets.mnist.load_data()

#Split the dataset 
(train_images,train_labels),(test_images,test_labels) = mnist

#Standardize/ normalize the data 
train_images,test_images = train_images/255.0,test_images/255.0 

#Add a channels dimension 
train_images = train_images[..., tf.newaxis].astype('float32')
test_images = test_images[...,tf.newaxis].astype('float32')

In [ ]:
#Batch and shuffle the dataset 
train_dataset = tf.data.Dataset.from_tensor_slices((train_images,train_labels)).shuffle(10000).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(32)

In [ ]:
#Build the model using the subclassing API
class MyModel(tf.keras.models.Model):
  def __init__(self):
    super(MyModel,self).__init__()
    self.Conv1 = tf.keras.layers.Conv2D(64,(3,3),activation = "relu")
    self.MaxPool = tf.keras.layers.MaxPool2D((2,2))
    self.flatten = tf.keras.layers.Flatten()
    self.Dense1 = tf.keras.layers.Dense(128,activation = "relu")
    self.Output = tf.keras.layers.Dense(10,)#activation ="softmax")
  
  def call(self,x):
    x = self.Conv1(x) 
    x = self.MaxPool(x)
    x = self.flatten(x)
    x = self.Dense1(x)
    x = self.Output(x)
    return x

In [ ]:
model = MyModel()

In [ ]:
#Define the hyperparameters
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = tf.keras.metrics.Accuracy()
optimizer = tf.keras.optimizers.Adam()
batch_size = 32

Selects the metrics to measure the loss and accuracy of the model

In [ ]:
#Metrics to measure the loss and accuracy of the model
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = "train_accuracy")

#test loss and accuracy 
test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test-accuracy")

In [ ]:
#Using tf.Gradient Tape to Train the model 
@tf.function #Converts a regular python function to a tensorflow graph function 
def train_step(images,labels):
  with tf.GradientTape() as tape:
    #Training = true is there are  layers with different behaviour during training versus inference such as Dropout
    predictions = model(images, training =True)
    #Calculate the loss 
    loss = loss_fn(labels,predictions)
    gradient = tape.gradient(loss,model.trainable_variables) #Get the gradient of the loss function 
    optimizer.apply_gradients(zip(gradient, model.trainable_variables)) #Update the parameters based on the gradient of the loss 

    train_loss(loss)
    train_accuracy(labels,predictions)

Test the model

In [ ]:
@tf.function
def test_step(images, labels):
  #Training = false is only needed if there are layers with different behaviours
  #during training versus inference 
  predictions = model(images,training=False)
  t_loss = loss_fn(predictions,labels)  

  test_loss(t_loss)
  test_accuracy(labels,predictions)

In [ ]:
EPOCHS = 10 

for epoch in range(EPOCHS):
  #Reset the metrics at the start of the next epoch 
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  #for image,labels in train_dataset:
   # train_step(image,labels)
  
  for test_images,test_labels in  test_dataset:
    test_step(test_images,test_labels)
    
  print(f"Epoch:{epoch + 1}/n",
        f"train_loss {train_loss.result()}/n",
        f"Accuracy{train_accuracy.result() * 100}/n",
        f"Test_loss {test_loss.result()}/n",
        f"Test_accuracy {test_accuracy.result()}/n"
        )
  #print(f"train_loss {train_loss.result()}")
  #print(f"")

ValueError: ignored

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.layers import Conv2D,Flatten,MaxPool2D,Dense,Dropout,BatchNormalization,Activation 
from tensorflow.nn import relu
from keras.callbacks import ModelCheckpoint
from tensorflow import  GradientTape

In [ ]:
#Load and split the dataset 
mnist = tf.keras.datasets.mnist.load_data()
(train_images,train_labels),(test_images,test_labels) = mnist

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
#Standardize the data/ normalize the data
train_images = train_images/255.0 #Convert the training data into floating point numbers in between zero and one( train X)
test_images = test_images/255.0 #Convert the testx into floating point numbers in between zero and one 

#Add a channel 
#train_images = tf.expand_dims(train_images, axis=1) 
#test_images= test_images[...,tf.newaxis].astype('float32')
#test_images = tf.expand_dims(test_images,axis=1)

#Add a channel
train_images = train_images[...,tf.newaxis].astype('float32')
test_images = test_images[...,tf.newaxis].astype('float32')

In [ ]:
train_images[0].shape

(28, 28, 1)

In [ ]:
#We batch and shuffle the dataset
train_ds = tf.data.Dataset.from_tensor_slices((train_images,train_labels)).shuffle(1000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((test_images,test_labels)).batch(32)

In [ ]:
#Build the networ using the  subclassing api
class MyModel(tf.keras.models.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    #First Convolution Block 
    self.Conv1 = Conv2D(16,(3,3),padding="same",activation =relu,input_shape=train_images[0].shape)
    self.MaxPool1 = MaxPool2D((2,2))
    self.Drop1 = Dropout(0.2) #Add a dropout layer to prevent overfitting
    #Second Convolution Block
    self.Conv2 = Conv2D(32,(3,3),padding="same",activation =relu)
    self.MaxPool2 = MaxPool2D((2,2))
    self.Drop2 = Dropout(0.2) #Another dropout layer to prevent overfitting
    
    #Third Convolution Block
    self.Conv3 = Conv2D(64,(3,3),padding="same",activation =relu)
    self.MaxPool3 = MaxPool2D((2,2))
    self.Drop3  =  Dropout(0.2) #Another dropout layer to prevent overfitting

    #Fourth Convolution Block
    self.Conv4 = Conv2D(128,(3,3),padding="same",activation=relu)
    self.MaxPool4 = MaxPool2D((2,2))
    self.Drop4 = Dropout(0.2)

    #Flatten out the learned features into a vector (1D array)
    self.flatten = Flatten()
    self.BatchNorm1 = BatchNormalization()
    self.Dense1 = Dense(300,kernel_initializer="he_normal")
    self.Activation1 = Activation('elu')
    self.Dense2 = Dense(100,kernel_initializer="he_normal")
    self.Activation2 = Activation('elu')
    self.BatchNorm2 = BatchNormalization()
    self.Output = Dense(10,activation = "softmax")

  #Calling Function that will be returned when the model will be called
  def call(self,x):
    #First ConvBlock
    x = self.Conv1(x)
    x = self.MaxPool1(x)
    x = self.Drop1(x)

    #Second Convolution Block
    x = self.Conv2(x)
    x = self.MaxPool2(x)
    x = self.Drop2(x)
    
    #Third Convolution Block
    x = self.Conv3(x)
    x = self.MaxPool3(x)
    #Third Convolution Block
    x = self.Conv3(x)
    x = self.Drop3(x)

    #Fourth Convolution Block
    x = self.Conv4(x)
    x = self.MaxPool4(x)
    x = self.Drop4(x)

    #FCN 
    x = self.flatten(x)
    x = self.BatchNorm1(x)
    x = self.Dense1(x)
    x = self.Activation1(x)
    x = self.Dense2(x)
    x = self.Activation2(x)
    x = self.BatchNorm2(x)
    Output = self.Output(x)
    return Output 

In [ ]:
#Create an instance of the model 
model = MyModel()

In [ ]:

#Define the hyperparameters and compile the model
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
learning_rate = 1e-3 
optimizer = tf.keras.optimizers.Adam()
batch_size = 64

#Select the metrics to measure the accuracy and loss of the model

#Training Metrics
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

#Test Metrics
test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")


#Cache prefetch and shuffle(Load the data into memory for faster loading after the first epoch)
#AUTOTUNE = tf.data.AUTOTUNE
#train_images = train_images().cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

In [ ]:
#Compile the model and define the training loop 
#model.compile(loss=loss_fn,optimizer=optimizer)

In [ ]:
#Define the callback function and save the model's weights after every five seconds
checkpoint = "expert_model/cp-{epoch:04d}.ckpt"
callback = ModelCheckpoint(checkpoint,verbose = 1,save_weights_only= True,save_freq=5 * batch_size)
model.save_weights(checkpoint.format(epoch=0))#String formating to save weights based on the epochs

In [ ]:
#Train the model
@tf.function
def train_step(image,label):
  with GradientTape() as tape:
    predictions = model(image)# make the predictions
    tr_loss = loss_fn(label,predictions) #Calculate the loss based on the predictions
    #Calculate the gradients based on the loss 
    gradients = tape.gradient(tr_loss, model.trainable_variables)
    #Update the parameters based on the gradients
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))

    train_loss(tr_loss)
    train_accuracy(label,predictions)

In [ ]:
#Testing the model
@tf.function 
def test_step(image, label):
  with GradientTape() as Tape:
    #Make the prediction
    predictions = model(image,training=False)#Since we are not going to update the weights we set the training attribute to false 
    t_loss = loss_fn(label, predictions)

    #Metrics of the loss and accuracy
    train_loss(t_loss)
    train_accuracy(label, predictions)

Training the model

In [ ]:
EPOCH = 10

for epoch in range(EPOCH):
  #RESET THE METRICS AFTER EVERY EPOCH
  train_loss.reset_states()
  train_accuracy.reset_states()

  #RESET THE TESTING METRICS
  test_loss.reset_states()
  test_accuracy.reset_states()

  #Iterate through the training data and call the training step function
  for image,labels in  train_ds:
    train_step(image,labels)
  for image,labels in test_ds:
    test_step(image,labels)

  print(f"EPOCH: {epoch + 1 }",
        f"Loss: {train_loss.result()}",
        f"Accuracy {train_accuracy.result() * 100}",
        f"Test_Loss: {test_loss.result()}",
        f"Test_Accuracy: {test_accuracy.result()*100}")
 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


EPOCH: 1 Loss: 0.12897785007953644 Accuracy 95.99142456054688 Test_Loss: 0.0 Test_Accuracy: 0.0
EPOCH: 2 Loss: 0.05096712335944176 Accuracy 98.38285827636719 Test_Loss: 0.0 Test_Accuracy: 0.0
EPOCH: 3 Loss: 0.03961346298456192 Accuracy 98.76142883300781 Test_Loss: 0.0 Test_Accuracy: 0.0
EPOCH: 4 Loss: 0.03221222758293152 Accuracy 99.05428314208984 Test_Loss: 0.0 Test_Accuracy: 0.0
EPOCH: 5 Loss: 0.027173558250069618 Accuracy 99.19285583496094 Test_Loss: 0.0 Test_Accuracy: 0.0
EPOCH: 6 Loss: 0.023702168837189674 Accuracy 99.24571228027344 Test_Loss: 0.0 Test_Accuracy: 0.0
EPOCH: 7 Loss: 0.022063596174120903 Accuracy 99.36714172363281 Test_Loss: 0.0 Test_Accuracy: 0.0
EPOCH: 8 Loss: 0.020552359521389008 Accuracy 99.37142944335938 Test_Loss: 0.0 Test_Accuracy: 0.0
EPOCH: 9 Loss: 0.01816730760037899 Accuracy 99.48999786376953 Test_Loss: 0.0 Test_Accuracy: 0.0
EPOCH: 10 Loss: 0.016401035711169243 Accuracy 99.52285766601562 Test_Loss: 0.0 Test_Accuracy: 0.0
